In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use Gemini 2.5 Pro to develop MCP Server



   
  
  <td style="text-align: center">
    <a href="https://github.com/wadave/vertex_ai_mcp_samples/blob/main/build_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Dave Wang](https://github.com/wadave) |

## Overview
The Model Context Protocol (MCP) is an open standard that simplifies how AI assistants connect with external data, tools, and systems. It achieves this by standardizing the way applications provide contextual information to Large Language Models (LLMs), creating a vital interface for models to interact directly with various external services.

Developers building MCP-enabled applications have the flexibility to utilize existing third-party MCP servers or implement their own custom server solutions.

This notebook focuses on the latter, demonstrating how to build custom MCP servers using Gemini 2.5 Pro. We will walk through code generation and testing for four specific examples:

#### MCP server code generation:
- Example 1: Building a BigQuery MCP Server
- Example 2: Building a MedlinePlus MCP Server
- Example 3: Building an NIH MCP Server
- Exmaple 4: Building a Cocktail MCP Server

#### MCP server code testing:
- Option 1: Use LangChain MCP Adaptor (working in Jupyter Notebook only, not Colab)
- Option 2: Build your own agent
- Option 3: Use Google ADK 

## Get started

### Install Google Gen AI SDK and other required packages


In [1]:
%pip install --upgrade --quiet google-genai google-cloud-secret-manager mcp geopy black google-cloud-bigquery langchain-mcp-adapters langchain langchain-google-vertexai langgraph google-adk

Note: you may need to restart the kernel to use updated packages.


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import Libraries

In [1]:
import sys
import os
import asyncio
import datetime
import json
import requests
import black
import re

from typing import Union, Dict, List, Optional
from google.genai.types import (
    GenerateContentConfig,
)
from google import genai
from google.genai import types
from google.cloud import aiplatform
from typing import List, Dict, Any

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_google_vertexai import ChatVertexAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from IPython.display import display, Markdown
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

### Helper function


In [2]:
def get_url_content(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Raise an exception if the request returned an error status code (like 404 or 500)
        response.raise_for_status()

        # Get the content of the response as text (HTML, in this case)
        # 'requests' automatically decodes the content based on HTTP headers
        file_content = response.text

        # Now you can work with the content
        print("Successfully fetched content")
        return file_content
        # Or, save it to a file:
        # with open("server_page.html", "w", encoding="utf-8") as f:
        #     f.write(file_content)
        # print("Content saved to server_page.html")

    except requests.exceptions.RequestException as e:
        # Handle potential errors during the request (e.g., network issues, DNS errors)
        print(f"Error fetching URL {url}: {e}")
    except requests.exceptions.HTTPError as e:
        # Handle HTTP error responses (e.g., 404 Not Found, 503 Service Unavailable)
        print(f"HTTP Error for {url}: {e}")


def format_python(raw_code, output_filename):

    try:
        # Format the code string using black
        # Use default FileMode which is generally recommended
        formatted_code = black.format_str(raw_code, mode=black.FileMode())

        # Save the formatted code to the specified file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(formatted_code)

        print(f"Successfully formatted the code and saved it to '{output_filename}'")

    except black.InvalidInput as e:
        print(
            f"Error formatting code: The input string does not seem to be valid Python syntax."
        )
        print(f"Details: {e}")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


def extract_json_from_string(input_str: str) -> Optional[Union[Dict, List]]:
    """
    Extracts JSON data from a string, handling potential variations.

    This function attempts to find JSON data within a string. It specifically
    looks for JSON enclosed in Markdown-like code fences (```json ... ```).
    If such a block is found, it extracts and parses the content.
    If no code block is found, it attempts to parse the entire input string
    as JSON.

    Args:
        input_str: The string potentially containing JSON data. It might be
                   a plain JSON string or contain a Markdown code block
                   with JSON, possibly preceded by other text (like 'shame').

    Returns:
        The parsed JSON object (typically a dictionary or list) if valid
        JSON is found and successfully parsed.
        Returns None if no valid JSON is found, if parsing fails, or if the
        input is not a string.
    """
    if not isinstance(input_str, str):
        # Handle cases where input is not a string
        return None

    # Pattern to find JSON within ```json ... ``` blocks
    # - ````json`: Matches the start fence.
    # - `\s*`: Matches any leading whitespace after the fence marker.
    # - `(.*?)`: Captures the content (non-greedily) between the fences. This is group 1.
    # - `\s*`: Matches any trailing whitespace before the end fence.
    # - ` ``` `: Matches the end fence.
    # - `re.DOTALL`: Allows '.' to match newline characters.
    pattern = r"```json\s*(.*?)\s*```"
    match = re.search(pattern, input_str, re.DOTALL)

    json_string_to_parse = None

    if match:
        # If a markdown block is found, extract its content
        json_string_to_parse = match.group(
            1
        ).strip()  # Get captured group and remove surrounding whitespace
    else:
        # If no markdown block, assume the *entire* input might be JSON
        # We strip whitespace in case the string is just JSON with padding
        json_string_to_parse = input_str.strip()

    if not json_string_to_parse:
        # If after stripping, the potential JSON string is empty, return None
        return None

    try:
        # Attempt to parse the determined string (either from block or whole input)
        parsed_json = json.loads(json_string_to_parse)
        return parsed_json
    except json.JSONDecodeError:
        # Parsing failed, indicating the string wasn't valid JSON
        return None
    except Exception as e:
        # Catch other potential unexpected errors during parsing
        print(f"An unexpected error occurred during JSON parsing: {e}")
        return None


from pathlib import Path


def create_folder_if_not_exists(folder_path_str: str) -> bool:
    """
    Creates a folder (and any necessary parent folders) if it doesn't already exist.
    Uses print() for status and error messages.

    Args:
        folder_path_str (str): The path string for the folder to be created.
                               Can be relative or absolute.

    Returns:
        bool: True if the folder already exists or was successfully created,
              False if an error occurred during creation (e.g., permission denied).
    """
    try:
        # Convert the string path to a Path object
        folder_path = Path(folder_path_str)

        # Use mkdir() with options:
        # parents=True: Creates any necessary parent directories. Like 'mkdir -p'.
        # exist_ok=True: Doesn't raise an error if the directory already exists.
        folder_path.mkdir(parents=True, exist_ok=True)

        # Print confirmation (using resolve() to show the absolute path)
        print(f"Info: Successfully ensured folder exists: {folder_path.resolve()}")
        return True

    except PermissionError:
        print(
            f"Error: Permission denied: Could not create folder at '{folder_path_str}'."
        )
        return False
    except OSError as e:
        # Catch other OS-related errors (e.g., path is a file, invalid path format on Windows)
        print(f"Error: OS error creating folder '{folder_path_str}': {e}")
        return False
    except Exception as e:
        # Catch any other unexpected errors
        print(
            f"Error: An unexpected error occurred creating folder '{folder_path_str}': {e}"
        )
        return False

In [3]:
create_folder_if_not_exists("server")

Info: Successfully ensured folder exists: /usr/local/google/home/wangdave/remote_ws/projects/gitlab_vertexai_mcp_samples/server


True

### Option 1 use a  Vertex AI project

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [24]:
# Use the environment variable if the user doesn't provide Project ID.
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Option 2. Use a Google API Key (Express Mode)
Uncomment the following block to use Express Mode

In [ ]:
# API_KEY = "[your-api-key]"  # @param {type: "string", placeholder: "[your-api-key]", isTemplate: true}

# if not API_KEY or API_KEY == "[your-api-key]":
#     raise Exception("You must provide an API key to use Vertex AI in express mode.")

# client = genai.Client(vertexai=True, api_key=API_KEY)

## Set up model id

In [4]:
MODEL_ID = "gemini-2.5-pro-exp-03-25"

### Get system instruction context info

In [5]:
# The URL you want to fetch
url = "https://modelcontextprotocol.io/quickstart/server"
reference_content = get_url_content(url)

Successfully fetched content


### Set up system instruction

In [6]:
from pydantic import BaseModel


class ResponseSchema(BaseModel):
    python_code: str
    description: str


system_instruction = f"""
  You are an MCP server export.
  Your mission is to write python code for MCP server.
  Here's the MCP server development guide and example
  {reference_content}
  
"""

#### Set function to generate MCP server code

In [7]:
def generate_mcp_server(prompt):
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=ResponseSchema,
        ),
    )

    return response.text

## Generate MCP Server Code

### Example 1:  Build MCP Server for Google Cloud BigQuery

In [20]:
prompt = """
  Please create an MCP server code for google cloud big query. It has two tools. One is to list tables for all datasets, the other is to describe a table. Google cloud project id and location will be provided in the query string. please use project id to access BigQuery client.
  Please output JSON output only.
  
"""

In [21]:
response_text = generate_mcp_server(prompt)

In [22]:
python_code = extract_json_from_string(response_text)["python_code"]
format_python(python_code, "server/bq.py")

Successfully formatted the code and saved it to 'server/bq.py'


### Example 2:  Build MCP server for Medlineplus website
Create an MCP server for 
https://medlineplus.gov/about/developers/webservices/ API service

In [12]:
med_url = "https://medlineplus.gov/about/developers/webservices/"
med_prompt_base = """
  Please create an MCP server code for https://medlineplus.gov/about/developers/webservices/. It has one tool, get_medical_term. You provide a medical term, this tool will return explanation of the medial term.
  
  Here's the API details:
  
"""

prompt = [med_prompt_base, types.Part.from_uri(file_uri=med_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/med.py")

Successfully formatted the code and saved it to 'server/med.py'


### Example 3: Build MCP Server for NIH

In [13]:
nih_url = "https://clinicaltables.nlm.nih.gov/apidoc/icd10cm/v3/doc.html"
nih_prompt_base = """
  Please create an MCP server code for NIH. It has one tool, get_icd_10_code. You provide a name or code, it will return top 5 results. 
  
  Here's the API details:

"""
prompt = [nih_prompt_base, types.Part.from_uri(file_uri=nih_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/nih.py")

Successfully formatted the code and saved it to 'server/nih.py'


### Example 4: Build MCP Server for the Cocktail DB

In [18]:
ct_url = "https://www.thecocktaildb.com/api.php"
ct_prompt_base = """
  Please create an MCP server code for the cocktail db. It has 5 tools:
  1. search cocktail by name
  2. list all cocktail by first letter
  3. search ingredient by name. 
  4. list random cocktails
  5. lookup full cocktail details by id
  
  Here's the API details:

"""
prompt = [ct_prompt_base, types.Part.from_uri(file_uri=ct_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/cocktail.py")

Successfully formatted the code and saved it to 'server/cocktail.py'


## Testing MCP Servers

### Option 1: Use LangChain MCP adaptor to test MCP servers 
It works in Jupyter Notebook only, not working in Colab. 

In [14]:
aiplatform.init(
    project=PROJECT_ID,
    location=LOCATION,
)

In [15]:
llm = ChatVertexAI(
    model="gemini-2.5-pro-exp-03-25",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
)

In [16]:
server_configs = {
    "nih": {
        "command": "python",
        "args": ["./server/nih.py"],
        "transport": "stdio",
    },
    "med": {
        "command": "python",
        "args": ["./server/med.py"],
        "transport": "stdio",
    },
    "bq": {
        "command": "python",
        "args": ["./server/bq.py"],
        "transport": "stdio",
    },
    "cocktail": {
        "command": "python",
        "args": ["./server/cocktail.py"],
        "transport": "stdio",
    },
}

Set up MCP Client using LangChain MCP Adaptor 

In [17]:
async def run_lc_react_agent(server_configs, message):
    async with MultiServerMCPClient(server_configs) as client:
        agent = create_react_agent(llm, client.get_tools())

        agent_response = await agent.ainvoke({"messages": message})
        for response in agent_response["messages"]:
            user = ""

            if isinstance(response, HumanMessage):
                user = "[User]"
            elif isinstance(response, ToolMessage):
                user = "-Tool-"
            elif isinstance(response, AIMessage):
                user = "[Agent]"

            if isinstance(response.content, list):
                display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
                continue
            display(Markdown(f"{user}: {response.content}"))

#### Example 1: BigQuery MCP server testing

In [23]:
await run_lc_react_agent(
    server_configs,
    "Please list my bigquery tables for project 'dw-genai-dev', the location is 'us'",
)

[User]: Please list my bigquery tables for project 'dw-genai-dev', the location is 'us'

[Agent]: 

-Tool-: Datasets and tables in project 'dw-genai-dev':

Dataset: demo_dataset1
  - Table: item_table
  - Table: user_table

Dataset: demo_dataset2
  - Table: item_table
  - Table: user_table

[Agent]: Okay, I can help with that. Here are the datasets and tables in the Google Cloud project 'dw-genai-dev' located in 'us':

**Dataset: demo_dataset1**
*   Table: item_table
*   Table: user_table

**Dataset: demo_dataset2**
*   Table: item_table
*   Table: user_table

#### Example 2: MedlinePlus MCP server testing

In [24]:
await run_lc_react_agent(server_configs, "Please explain flu in details")

[User]: Please explain flu in details

[Agent]: 

-Tool-: Failed to parse XML, but got a response (may contain errors or be HTML): <?xml version="1.0" encoding="UTF-8"?>
<nlmSearchResult>
  <term>flu</term>
  <file>viv_Vc6q5A</file>
  <server>pvlb7srch16</server>
  <count>67</count>
  <retstart>0</retstart>
  <retmax>10</retmax>
  <list num="67" start="0" per="10">
    <document rank="2" url="https://medlineplus.gov/flu.html">
      <content name="healthTopic">
        <health-topic meta-desc="The flu can cause serious illness in people over 65, babies, and people with certain chronic illnesses. The best way to prevent the ...

[Agent]: The flu, or influenza, is a contagious respiratory illness caused by influenza viruses. It can cause mild to severe illness, and at times can lead to serious complications.

Key points about the flu:
*   **Symptoms:** Often come on suddenly and can include fever or feeling feverish/chills, cough, sore throat, runny or stuffy nose, muscle or body aches, headaches, and fatigue (tiredness). Some people may have vomiting and diarrhea, though this is more common in children than adults.
*   **Transmission:** Flu viruses spread mainly by tiny droplets made when people with flu cough, sneeze or talk. These droplets can land in the mouths or noses of people who are nearby. Less often, a person might get flu by touching a surface or object that has flu virus on it and then touching their own mouth, nose, or possibly their eyes.
*   **Complications:** Can include pneumonia, bronchitis, sinus infections, and ear infections. Flu can also worsen chronic health problems like asthma or heart failure. Certain groups are at higher risk for serious complications, including older adults (65+), young children, pregnant women, and people with certain chronic health conditions (like asthma, diabetes, or heart disease).
*   **Prevention:** The best way to prevent the flu is by getting a flu vaccine each year. Good health habits like covering your cough, washing your hands often, and avoiding close contact with sick people can also help stop the spread of germs.
*   **Treatment:** Most people with the flu have mild illness and do not need medical care or antiviral drugs. If you get sick with flu symptoms, in most cases, you should stay home and avoid contact with other people except to get medical care. If you have symptoms of flu and are in a high-risk group, or are very sick or worried about your illness, contact your healthcare provider. Antiviral drugs can be used to treat flu illness and can shorten the time you are sick and prevent serious complications.

For more detailed information, you can visit MedlinePlus: [https://medlineplus.gov/flu.html](https://medlineplus.gov/flu.html)

#### Example 3: NIH MCP Server testing

In [25]:
await run_lc_react_agent(server_configs, "can you tell me icd-10 code for influenza A?")

[User]: can you tell me icd-10 code for influenza A?

[Agent]: 

-Tool-: Found 8 results. Top 5 matching 'influenza A':

Code: J09.X1
Name: Influenza due to identified novel influenza A virus with pneumonia
---
Code: J09.X3
Name: Influenza due to identified novel influenza A virus with gastrointestinal manifestations
---
Code: J09.X9
Name: Influenza due to identified novel influenza A virus with other manifestations
---
Code: J09.X2
Name: Influenza due to identified novel influenza A virus with other respiratory manifestations
---
Code: A41.3
Name: Sepsis due to Hemophilus influenzae

[Agent]: Okay, I found several ICD-10 codes related to "influenza A". The top results are:

*   **J09.X1**: Influenza due to identified novel influenza A virus with pneumonia
*   **J09.X3**: Influenza due to identified novel influenza A virus with gastrointestinal manifestations
*   **J09.X9**: Influenza due to identified novel influenza A virus with other manifestations
*   **J09.X2**: Influenza due to identified novel influenza A virus with other respiratory manifestations

There's also **A41.3** for Sepsis due to *Hemophilus influenzae*, but the J09 codes seem more specific to Influenza A virus infection.

The specific code would depend on the patient's presentation and manifestations.

### Example 4: Cocktail MCP server testing

In [26]:
await run_lc_react_agent(server_configs, "Please get full detail of cocktail margarita")

[User]: Please get full detail of cocktail margarita

[Agent]: Okay, I can help with that. First, I need to find the ID for "Margarita".

-Tool-: Found cocktails:
---
ID: 11007
Name: Margarita
Category: Ordinary Drink
Glass: Cocktail glass
Alcoholic: Alcoholic
Instructions: Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only the outer rim and sprinkle the salt on it. The sa...
Thumbnail: https://www.thecocktaildb.com/images/media/drink/5noda61589575158.jpg
---
ID: 11118
Name: Blue Margarita
Category: Ordinary Drink
Glass: Cocktail glass
Alcoholic: Alcoholic
Instructions: Rub rim of cocktail glass with lime juice. Dip rim in coarse salt. Shake tequila, blue curacao, and lime juice with ice, strain into the salt-rimmed g...
Thumbnail: https://www.thecocktaildb.com/images/media/drink/bry4qh1582751040.jpg
---
ID: 17216
Name: Tommy's Margarita
Category: Ordinary Drink
Glass: Old-Fashioned glass
Alcoholic: Alcoholic
Instructions: Shake and strain into a chilled cocktail glass....
Thumbnail: https://www.thecocktaildb.com/images/media/drink/loezxn1504373874.jpg
---
ID: 16158
Name: Whitecap Margarita
Category: Other / Unknown
Glass: Margarita/Coupette glass
Alcoholic: Alcoholic
Instructions: Place all ingredients in a blender and blend until smooth. This makes one drink....
Thumbnail: https://www.thecocktaildb.com/images/media/drink/srpxxp1441209622.jpg
---
ID: 12322
Name: Strawberry Margarita
Category: Ordinary Drink
Glass: Cocktail glass
Alcoholic: Alcoholic
Instructions: Rub rim of cocktail glass with lemon juice and dip rim in salt. Shake schnapps, tequila, triple sec, lemon juice, and strawberries with ice, strain in...
Thumbnail: https://www.thecocktaildb.com/images/media/drink/tqyrpw1439905311.jpg
---
ID: 178332
Name: Smashed Watermelon Margarita
Category: Cocktail
Glass: Collins glass
Alcoholic: Alcoholic
Instructions: In a mason jar muddle the watermelon and 5 mint leaves together into a puree and strain. Next add the grapefruit juice, juice of half a lime and the t...
Thumbnail: https://www.thecocktaildb.com/images/media/drink/dztcv51598717861.jpg

[Agent]: Okay, I found the classic Margarita with ID `11007`. Now I'll get the full details for you.

-Tool-: ID: 11007
Name: Margarita
Alternate Name: None
Tags: IBA,ContemporaryClassic
Category: Ordinary Drink
IBA Category: Contemporary Classics
Alcoholic: Alcoholic
Glass: Cocktail glass
Instructions: Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only the outer rim and sprinkle the salt on it. The salt should present to the lips of the imbiber and never mix into the cocktail. Shake the other ingredients with ice, then carefully pour into the glass.

Ingredients:
- 1 1/2 oz Tequila
- 1/2 oz Triple sec
- 1 oz Lime juice
-  Salt

Image URL: https://www.thecocktaildb.com/images/media/drink/5noda61589575158.jpg
Last Modified: 2015-08-18 14:42:59

[Agent]: Here are the full details for the Margarita cocktail:

**ID:** 11007
**Name:** Margarita
**Alternate Name:** None
**Tags:** IBA, ContemporaryClassic
**Category:** Ordinary Drink
**IBA Category:** Contemporary Classics
**Alcoholic:** Alcoholic
**Glass:** Cocktail glass
**Instructions:** Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only the outer rim and sprinkle the salt on it. The salt should present to the lips of the imbiber and never mix into the cocktail. Shake the other ingredients with ice, then carefully pour into the glass.

**Ingredients:**
*   1 1/2 oz Tequila
*   1/2 oz Triple sec
*   1 oz Lime juice
*   Salt

**Image URL:** [https://www.thecocktaildb.com/images/media/drink/5noda61589575158.jpg](https://www.thecocktaildb.com/images/media/drink/5noda61589575158.jpg)
**Last Modified:** 2015-08-18 14:42:59

#### Agent streaming

In [21]:
async def run_streaming_agent():
    async with MultiServerMCPClient(
        {
            "nih": {
                "command": "python",
                "args": ["./server/nih.py"],
                "transport": "stdio",
            },
        }
    ) as client:
        agent = create_react_agent(llm, client.get_tools())
        # Initialize conversation history using simple tuples
        inputs = {"messages": []}

        print("Agent is ready. Type 'exit' to quit.")
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Exiting chat.")
                break

            # Append user message to history
            inputs["messages"].append(("user", user_input))

            # call our graph with streaming to see the steps
            async for state in agent.astream(inputs, stream_mode="values"):
                last_message = state["messages"][-1]
                print(last_message)
                last_message.pretty_print()

            # update the inputs with the agent's response
            inputs["messages"] == state["messages"]

In [45]:
await run_streaming_agent()

Agent is ready. Type 'exit' to quit.
content="what's icd-10 code for flu A?" additional_kwargs={} response_metadata={} id='49034228-78fb-41ed-af5a-9f3de1bc630b'
================================ Human Message =================================

what's icd-10 code for flu A?
content='' additional_kwargs={'function_call': {'name': 'get_icd_10_code', 'arguments': '{"query": "flu A"}'}} response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 73, 'candidates_token_count': 12, 'total_token_count': 236, 'prompt_tokens_details': [{'modality': 1, 'token_count': 73}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 12}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.6214426755905151, 'model_name': 'gemini-2.5-pro-exp-03-25'} id='run-aec0701c-13f1-4c98-9fc1-3b1809d87c91-0' tool_calls=[{'name': 'get_icd_10_code', 'args': {'query': 'flu A'}, 'id': '7d2f26c9-805f-49d5-886b-e9db51b92f

### Option 2: Build your own agent to test MCP servers


##### Gemini Agent

Within an MCP client session, this agent loop runs a multi-turn conversation loop with a Gemini model, handling tool calls via MCP server.

This function orchestrates the interaction between a user prompt, a Gemini model capable of function calling, and a session object that provides and executes tools. It handles the cycle of:
-  Gemini gets tool information from MCP client session
-  Sending the user prompt (and conversation history) to the model.
-  If the model requests tool calls, Gemini makes initial function calls to get structured data as per schema, and 
-  Sending the tool execution results back to the model.
-  Repeating until the model provides a text response or the maximum number of tool execution turns is reached.
-  Gemini generates final response based on tool responses and original query.
  
MCP integration with Gemini

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/mcp/mcp_tool_call.png" alt="MCP with Gemini" height="700">

In [8]:
# --- Configuration ---
# Consider using a more recent/recommended model if available and suitable

DEFAULT_MAX_TOOL_TURNS = 5  # Maximum consecutive turns for tool execution
DEFAULT_INITIAL_TEMPERATURE = (
    0.0  # Temperature for the first LLM call (more deterministic)
)
DEFAULT_TOOL_CALL_TEMPERATURE = (
    1.0  # Temperature for LLM calls after tool use (potentially more creative)
)

# Make tool calls via MCP Server


async def _execute_tool_calls(
    function_calls: List[types.FunctionCall], session: ClientSession
) -> List[types.Part]:
    """
    Executes a list of function calls requested by the Gemini model via the session.

    Args:
        function_calls: A list of FunctionCall objects from the model's response.
        session: The session object capable of executing tools via `call_tool`.

    Returns:
        A list of Part objects, each containing a FunctionResponse corresponding
        to the execution result of a requested tool call.
    """
    tool_response_parts: List[types.Part] = []
    print(f"--- Executing {len(function_calls)} tool call(s) ---")

    for func_call in function_calls:
        tool_name = func_call.name
        # Ensure args is a dictionary, even if missing or not a dict type
        args = func_call.args if isinstance(func_call.args, dict) else {}
        print(f"  Attempting to call session tool: '{tool_name}' with args: {args}")

        tool_result_payload: Dict[str, Any]
        try:
            # Execute the tool using the provided session object
            # Assumes session.call_tool returns an object with attributes
            # like `isError` (bool) and `content` (list of Part-like objects).
            tool_result = await session.call_tool(tool_name, args)
            print(f"  Session tool '{tool_name}' execution finished.")

            # Extract result or error message from the tool result object
            result_text = ""
            # Check structure carefully based on actual `session.call_tool` return type
            if (
                hasattr(tool_result, "content")
                and tool_result.content
                and hasattr(tool_result.content[0], "text")
            ):
                result_text = tool_result.content[0].text or ""

            if hasattr(tool_result, "isError") and tool_result.isError:
                error_message = (
                    result_text
                    or f"Tool '{tool_name}' failed without specific error message."
                )
                print(f"  Tool '{tool_name}' reported an error: {error_message}")
                tool_result_payload = {"error": error_message}
            else:
                print(
                    f"  Tool '{tool_name}' succeeded. Result snippet: {result_text[:150]}..."
                )  # Log snippet
                tool_result_payload = {"result": result_text}

        except Exception as e:
            # Catch exceptions during the tool call itself
            error_message = f"Tool execution framework failed: {type(e).__name__}: {e}"
            print(f"  Error executing tool '{tool_name}': {error_message}")
            tool_result_payload = {"error": error_message}

        # Create a FunctionResponse Part to send back to the model
        tool_response_parts.append(
            types.Part.from_function_response(
                name=tool_name, response=tool_result_payload
            )
        )
    print(f"--- Finished executing tool call(s) ---")
    return tool_response_parts


async def run_agent_loop(
    prompt: str,
    client: genai.Client,
    session: ClientSession,
    model_id: str = MODEL_ID,
    max_tool_turns: int = DEFAULT_MAX_TOOL_TURNS,
    initial_temperature: float = DEFAULT_INITIAL_TEMPERATURE,
    tool_call_temperature: float = DEFAULT_TOOL_CALL_TEMPERATURE,
) -> types.GenerateContentResponse:
    """
    Runs a multi-turn conversation loop with a Gemini model, handling tool calls.

    This function orchestrates the interaction between a user prompt, a Gemini
    model capable of function calling, and a session object that provides
    and executes tools. It handles the cycle of:
    1. Sending the user prompt (and conversation history) to the model.
    2. If the model requests tool calls, executing them via the `session`.
    3. Sending the tool execution results back to the model.
    4. Repeating until the model provides a text response or the maximum
       number of tool execution turns is reached.

    Args:
        prompt: The initial user prompt to start the conversation.
        client: An initialized Gemini GenerativeModel client object

        session: An active session object responsible for listing available tools
                 via `list_tools()` and executing them via `call_tool(tool_name, args)`.
                 It's also expected to have an `initialize()` method.
        model_id: The identifier of the Gemini model to use (e.g., "gemini-1.5-pro-latest").
        max_tool_turns: The maximum number of consecutive turns dedicated to tool calls
                        before forcing a final response or exiting.
        initial_temperature: The temperature setting for the first model call.
        tool_call_temperature: The temperature setting for subsequent model calls
                               that occur after tool execution.

    Returns:
        The final Response from the Gemini model after the
        conversation loop concludes (either with a text response or after
        reaching the max tool turns).

    Raises:
        ValueError: If the session object does not provide any tools.
        Exception: Can potentially raise exceptions from the underlying API calls
                   or session tool execution if not caught internally by `_execute_tool_calls`.
    """
    print(
        f"Starting agent loop with model '{model_id}' and prompt: '{prompt[:100]}...'"
    )

    # Initialize conversation history with the user's prompt
    contents: List[types.Content] = [
        types.Content(role="user", parts=[types.Part(text=prompt)])
    ]

    # Ensure the session is ready (if needed)
    if hasattr(session, "initialize") and callable(session.initialize):
        print("Initializing session...")
        await session.initialize()
    else:
        print("Session object does not have an initialize() method, proceeding anyway.")

    # --- 1. Discover Tools from Session ---
    print("Listing tools from session...")
    # Assumes session.list_tools() returns an object with a 'tools' attribute (list)
    # Each item in the list should have 'name', 'description', and 'inputSchema' attributes.
    session_tool_list = await session.list_tools()

    if not session_tool_list or not session_tool_list.tools:
        raise ValueError("No tools provided by the session. Agent loop cannot proceed.")

    # Convert session tools to the format required by the Gemini API
    gemini_tool_config = types.Tool(
        function_declarations=[
            types.FunctionDeclaration(
                name=tool.name,
                description=tool.description,
                parameters=tool.inputSchema,  # Assumes inputSchema is compatible
            )
            for tool in session_tool_list.tools
        ]
    )
    print(
        f"Configured Gemini with {len(gemini_tool_config.function_declarations)} tool(s)."
    )

    # --- 2. Initial Model Call ---
    print("Making initial call to Gemini model...")
    current_temperature = initial_temperature
    response = await client.aio.models.generate_content(
        model=MODEL_ID,
        contents=contents,  # Send updated history
        config=types.GenerateContentConfig(
            temperature=1.0,
            tools=[gemini_tool_config],
        ),  # Keep sending same config
    )
    print("Initial response received.")

    # Append the model's first response (potentially including function calls) to history
    # Need to handle potential lack of candidates or content
    if not response.candidates:
        print("Warning: Initial model response has no candidates.")
        # Decide how to handle this - raise error or return the empty response?
        return response
    contents.append(response.candidates[0].content)

    # --- 3. Tool Calling Loop ---
    turn_count = 0
    # Check specifically for FunctionCall objects in the latest response part
    latest_content = response.candidates[0].content
    has_function_calls = any(part.function_call for part in latest_content.parts)

    while has_function_calls and turn_count < max_tool_turns:
        turn_count += 1
        print(f"\n--- Tool Turn {turn_count}/{max_tool_turns} ---")

        # --- 3.1 Execute Pending Function Calls ---
        function_calls_to_execute = [
            part.function_call for part in latest_content.parts if part.function_call
        ]
        tool_response_parts = await _execute_tool_calls(
            function_calls_to_execute, session
        )

        # --- 3.2 Add Tool Responses to History ---
        # Send back the results for *all* function calls from the previous turn
        contents.append(
            types.Content(role="function", parts=tool_response_parts)
        )  # Use "function" role
        print(f"Added {len(tool_response_parts)} tool response part(s) to history.")

        # --- 3.3 Make Subsequent Model Call with Tool Responses ---
        print("Making subsequent API call to Gemini with tool responses...")
        current_temperature = tool_call_temperature  # Use different temp for follow-up
        response = await client.aio.models.generate_content(
            model=MODEL_ID,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[gemini_tool_config],
            ),
        )
        print("Subsequent response received.")

        # --- 3.4 Append latest model response and check for more calls ---
        if not response.candidates:
            print("Warning: Subsequent model response has no candidates.")
            break  # Exit loop if no candidates are returned
        latest_content = response.candidates[0].content
        contents.append(latest_content)
        has_function_calls = any(part.function_call for part in latest_content.parts)
        if not has_function_calls:
            print(
                "Model response contains text, no further tool calls requested this turn."
            )

    # --- 4. Loop Termination Check ---
    if turn_count >= max_tool_turns and has_function_calls:
        print(
            f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop even though function calls might be pending."
        )
    elif not has_function_calls:
        print("Tool calling loop finished naturally (model provided text response).")

    # --- 5. Return Final Response ---
    print("Agent loop finished. Returning final response.")
    return response

#### Set up MCP client

In [9]:
async def run_simple_agent(server_params, query):
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = query
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await run_agent_loop(prompt, client, session)
            return res

In [10]:
bq_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/bq.py"],
)

In [11]:
med_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/med.py"],
)

In [12]:
nih_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/nih.py"],
)

In [13]:
ct_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/cocktail.py"],
)

In [14]:
bq_query = (
    "Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'"
)
bq_res = await run_simple_agent(bq_server_params, bq_query)
print(bq_res.text)

Running agent loop with prompt: Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'...'
Initializing session...
Listing tools from session...
Configured Gemini with 2 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'list_tables' with args: {'location': 'us', 'project_id': 'dw-genai-dev'}
  Session tool 'list_tables' execution finished.
  Tool 'list_tables' succeeded. Result snippet: demo_dataset1.item_table
demo_dataset1.user_table
demo_dataset2.item_table
demo_dataset2.user_table...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further too

In [53]:
med_query = "Please explain flu in detail."
med_res = await run_simple_agent(med_server_params, med_query)
print(med_res.text)

Running agent loop with prompt: Please explain flu in detail.
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please explain flu in detail....'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_medical_term' with args: {'term': 'flu'}
  Session tool 'get_medical_term' execution finished.
  Tool 'get_medical_term' succeeded. Result snippet: Term: Flu

Explanation: What is the flu? The flu , also called influenza , is a respiratory infection caused by viruses. Each year, millions of Americ...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calling loop finished naturally

In [54]:
nih_query = "Please tell me icd-10 code for pneumonia"
nih_res = await run_simple_agent(nih_server_params, nih_query)
print(nih_res.text)

Running agent loop with prompt: Please tell me icd-10 code for pneumonia
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please tell me icd-10 code for pneumonia...'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_icd_10_code' with args: {'query': 'pneumonia'}
  Session tool 'get_icd_10_code' execution finished.
  Tool 'get_icd_10_code' succeeded. Result snippet: Found 77 results. Showing top 5:
Code: A01.03, Name: Typhoid pneumonia
Code: A02.22, Name: Salmonella pneumonia
Code: A54.84, Name: Gonococcal pneumon...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calli

In [23]:
ct_query = "Please tell me the details of cocktail margarita"
bq_res = await run_simple_agent(ct_server_params, ct_query)
print(bq_res.text)

Running agent loop with prompt: Please tell me the details of cocktail margarita
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please tell me the details of cocktail margarita...'
Initializing session...
Listing tools from session...
Configured Gemini with 5 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'search_cocktail_by_name' with args: {'name': 'margarita'}
  Session tool 'search_cocktail_by_name' execution finished.
  Tool 'search_cocktail_by_name' succeeded. Result snippet: Found cocktails:
---
ID: 11007
Name: Margarita
Category: Ordinary Drink
Glass: Cocktail glass
Alcoholic: Alcoholic
Instructions: Rub the rim of the gl...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further too

### Option 3 Testing with Google ADK
Note: It works in  Jupyter Notebook only

In [15]:
from google.adk.tools.mcp_tool.mcp_toolset import (
    MCPToolset,
    SseServerParams,
    StdioServerParameters,
)

In [16]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

In [17]:
from google.adk.tools.mcp_tool.mcp_toolset import (
    MCPToolset,
    SseServerParams,
    StdioServerParameters,
)
from google.adk.agents.llm_agent import LlmAgent

import asyncio
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService

load_dotenv()


async def get_tools_async(server_params):
    """Gets tools from MCP Server."""
    tools, exit_stack = await MCPToolset.from_server(connection_params=server_params)
    # MCP requires maintaining a connection to the local MCP Server.
    # Using exit_stack to clean up server connection before exit.
    return tools, exit_stack


async def get_agent_async(server_params):
    """Creates an ADK Agent with tools from MCP Server."""
    tools, exit_stack = await get_tools_async(server_params)
    root_agent = LlmAgent(
        model=MODEL_ID,
        name="ai_assistant",
        instruction="Use tools to get information to answer user questions",
        tools=tools,
    )
    return root_agent, exit_stack

### An Agent built using Google ADK

In [18]:
async def run_adk_agent(server_params, question):
    session_service = InMemorySessionService()
    artifacts_service = InMemoryArtifactService()
    session = session_service.create_session(state={}, app_name="my_app", user_id="123")

    query = question
    print("[user]: ", query)
    content = types.Content(role="user", parts=[types.Part(text=query)])
    root_agent, exit_stack = await get_agent_async(server_params)
    runner = Runner(
        app_name="my_app",
        agent=root_agent,
        artifact_service=artifacts_service,
        session_service=session_service,
    )
    events_async = runner.run_async(
        session_id=session.id, user_id="123", new_message=content
    )
    # print(events_async)

    async for event in events_async:
        # print(event)
        if event.content.role == "user" and event.content.parts[0].text:
            print("[user]:", event.content.parts[0].text)

        if event.content.parts[0].function_response:
            print("[-tool_response-]", event.content.parts[0].function_response)

        if event.content.role == "model" and event.content.parts[0].text:
            print("[agent]:", event.content.parts[0].text)

    await exit_stack.aclose()

In [19]:
# For this server, make sure you have Node.js installed on your machine
bnb_server_params = StdioServerParameters(
    command="npx", args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"]
)

In [20]:
response = await run_adk_agent(
    bnb_server_params,
    "Please find a room in LA, CA, April 15, 2025, checkout date is april 18, 2 adults",
)

[user]:  Please find a room in LA, CA, April 15, 2025, checkout date is april 18, 2 adults


[-tool_response-] id='af-956a3d59-ffe0-44fc-b459-7ff78c91e2a6' name='airbnb_search' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='{\n  "searchUrl": "https://www.airbnb.com/s/LA%2C%20CA/homes?checkin=2025-04-15&checkout=2025-04-18&adults=2&children=0&infants=0&pets=0",\n  "searchResults": [\n    {\n      "url": "https://www.airbnb.com/rooms/51181269",\n      "listing": {\n        "id": "51181269",\n        "title": "Condo in Long Beach",\n        "structuredContent": {\n          "primaryLine": ""\n        }\n      },\n      "avgRatingA11yLabel": "4.82 out of 5 average rating,  141 reviews",\n      "structuredDisplayPrice": {\n        "primaryLine": {\n          "accessibilityLabel": "$386 for 3 nights, originally $506"\n        },\n        "explanationData": {\n          "title": "Price details",\n          "priceDetails": "$129 x 3 nights: $386, "\n        }\n      }\n    },\n    {\n      "url": "https://www.airbnb.com/rooms/32836965",\n      "l

In [21]:
await run_adk_agent(bnb_server_params, "give more details of id 7462294")

[user]:  give more details of id 7462294


[-tool_response-] id='af-ad06054b-64e7-4939-b65d-b53efe079830' name='airbnb_listing_details' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='{\n  "listingUrl": "https://www.airbnb.com/rooms/7462294?adults=1&children=0&infants=0&pets=0",\n  "details": [\n    {\n      "id": "LOCATION_DEFAULT",\n      "lat": 34.0743,\n      "lng": -118.176,\n      "subtitle": "Los Angeles, California, United States",\n      "title": "Where you’ll be"\n    },\n    {\n      "id": "POLICIES_DEFAULT",\n      "title": "Things to know",\n      "houseRulesSections": "Checking in and out: Check-in after 4:00\u202fPM, Checkout before 12:00\u202fPM, Self check-in with building staff, During your stay: 2 guests maximum, No pets, No parties or events, No smoking, Additional rules, Before you leave: Additional requests"\n    },\n    {\n      "id": "HIGHLIGHTS_DEFAULT",\n      "highlights": "Top 10% of homes, 24-hour self check-in, Mountain and valley views"\n    },\n    {\n      "

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Exiting chat.")
        break
    await run_adk_agent(bnb_server_params, user_input)

[user]:  Please find a room in LA, CA, April 15, 2025, checkout date is april 18, 2 adults


[-tool_response-] id='adk-5d99ae3f-df1a-46ea-b8fb-724af3802ce0' name='airbnb_search' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='{\n  "searchUrl": "https://www.airbnb.com/s/LA%2C%20CA/homes?checkin=2025-04-15&checkout=2025-04-18&adults=2&children=0&infants=0&pets=0",\n  "searchResults": [\n    {\n      "url": "https://www.airbnb.com/rooms/51181269",\n      "listing": {\n        "id": "51181269",\n        "title": "Condo in Long Beach",\n        "structuredContent": {\n          "primaryLine": ""\n        }\n      },\n      "avgRatingA11yLabel": "4.82 out of 5 average rating,  141 reviews",\n      "structuredDisplayPrice": {\n        "primaryLine": {\n          "accessibilityLabel": "$386 for 3 nights, originally $506"\n        },\n        "explanationData": {\n          "title": "Price details",\n          "priceDetails": "$129 x 3 nights: $386, "\n        }\n      }\n    },\n    {\n      "url": "https://www.airbnb.com/rooms/32836965",\n      "

[-tool_response-] id='adk-778f1133-9105-4982-9e32-8bdf94e5ab85' name='airbnb_listing_details' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='{\n  "listingUrl": "https://www.airbnb.com/rooms/51181269?adults=1&children=0&infants=0&pets=0",\n  "details": [\n    {\n      "id": "POLICIES_DEFAULT",\n      "title": "Things to know",\n      "houseRulesSections": "Checking in and out: Check-in after 4:00\u202fPM, Checkout before 11:00\u202fAM, Self check-in with smart lock, During your stay: 2 guests maximum, No pets, No parties or events, No smoking, Before you leave: Gather used towels, Turn things off, Lock up"\n    },\n    {\n      "id": "LOCATION_DEFAULT",\n      "lat": 33.7714,\n      "lng": -118.1738,\n      "subtitle": "Long Beach, California, United States",\n      "title": "Where you’ll be"\n    },\n    {\n      "id": "HIGHLIGHTS_DEFAULT",\n      "highlights": "Self check-in, Nicole is a Superhost"\n    },\n    {\n      "id": "DESCRIPTION_DEFAULT

In [22]:
ct_server_params = StdioServerParameters(
    command="python",
    args=["./server/cocktail2.py"],
)

In [23]:
await run_adk_agent(
    ct_server_params,
    "Please get cocktail margarita id and then full detail of cocktail margarita",
)

[user]:  Please get cocktail margarita id and then full detail of cocktail margarita


[-tool_response-] id='af-0e90b479-a41e-4ab0-9350-0758c3a70afd' name='search_cocktail_by_name' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text="Name: Margarita, ID: 11007\n  Instructions: Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only...\n  Glass: Cocktail glass\n  Type: Alcoholic\n---\nName: Blue Margarita, ID: 11118\n  Instructions: Rub rim of cocktail glass with lime juice. Dip rim in coarse salt. Shake tequila, blue curacao, and ...\n  Glass: Cocktail glass\n  Type: Alcoholic\n---\nName: Tommy's Margarita, ID: 17216\n  Instructions: Shake and strain into a chilled cocktail glass....\n  Glass: Old-Fashioned glass\n  Type: Alcoholic\n---\nName: Whitecap Margarita, ID: 16158\n  Instructions: Place all ingredients in a blender and blend until smooth. This makes one drink....\n  Glass: Margarita/Coupette glass\n  Type: Alcoholic\n---\nName: Strawberry Margarita, ID: 12322\n  Instructions: Rub rim o

[-tool_response-] id='af-4d4ebb21-3436-4d2c-adf9-09aa0bf2f9dd' name='lookup_cocktail_by_id' response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='Name: Margarita, ID: 11007\n  Instructions: Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only...\n  Glass: Cocktail glass\n  Type: Alcoholic', annotations=None)], isError=False)}
[agent]: Okay, I found the ID for the Margarita cocktail is 11007.

Here are the full details for the Margarita (ID: 11007):

*   **Name**: Margarita
*   **ID**: 11007
*   **Type**: Alcoholic
*   **Glass**: Cocktail glass
*   **Instructions**: Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only the outer rim and sprinkle the salt onto it. The salt should present to the lips of the imbiber and never mix into the cocktail. Shake the other ingredients with ice, then carefully pour into the glass.


### References:

https://modelcontextprotocol.io/introduction  

https://github.com/philschmid/gemini-samples/blob/main/examples/gemini-mcp-example.ipynb  

https://github.com/modelcontextprotocol/python-sdk